In [1]:
import pandas as pd
import numpy as np
import json
from polyglot.text import Text
from polyglot.downloader import downloader
from textblob import TextBlob
import nltk
import timestring
from datetime import datetime
import plotly.plotly as py
import plotly.graph_objs as go

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:318: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#snimissingwarning.
  SNIMissingWarning
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [23]:
tweets=pd.read_csv('/notebooks/datasets/facebook/allcomment.csv', index_col=0, encoding='utf8', engine='python')

In [24]:
tweets.shape

(1777029, 7)

In [25]:
tweets.head()

,id,post_id,page_id,fb_id,message,can_remove,created_time
0,1427636.0,71666378535,1.530806e+11,1.020608e+16,Already saw it live.. awesome as usual.. we ju...,0.0,2009-04-16 22:08:00
1,1427701.0,71666378535,1.530806e+11,1.015364e+16,Will grab a copy of his new book 'Think Like a...,0.0,2009-04-16 22:13:21
2,1427715.0,71666378535,1.530806e+11,1.010013e+16,His books are great. One of my idols.,0.0,2009-04-16 22:14:32
3,1427780.0,71666378535,1.530806e+11,1.015399e+16,Awesome even just from video..,0.0,2009-04-16 22:20:13
4,1427840.0,71666378535,1.530806e+11,1.015288e+16,have to see,0.0,2009-04-16 22:24:36


In [26]:
mapp = {153080620724: 'Donald Trump', 124955570892789: 'Bernie Sanders', 889307941125736: 'Hillary Clinton'}
def map_names(x):
    if x in mapp:
        return mapp[x]
    else:
        return 0

In [27]:
tweets['displayName'] = tweets.page_id.apply(lambda x: map_names(x))

In [28]:
tweets = tweets[tweets.displayName != 0]

In [29]:
tweets.shape

(1776766, 8)

In [30]:
tweets = tweets[['created_time', 'displayName', 'message']]

In [31]:
tweets.created_time.head(1)

0    2009-04-16 22:08:00
Name: created_time, dtype: object

In [32]:
tweets = tweets[~pd.isnull(tweets.created_time) & ~pd.isnull(tweets.message)]

In [33]:
# [x for x in tweets['created_time'] if not isinstance(x, str)]

In [34]:
tweets['postedTime'] = tweets.created_time.apply(
    lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [35]:
tweets = tweets[tweets.postedTime > datetime(2015, 1, 1, 1, 1, 1, 1)]

In [36]:
def get_entity_sentiment(e):
    try:
        pos=e.positive_sentiment
        neg=e.negative_sentiment
        return pos,neg
    except:
        return '',''
def get_sentiment(t):
    blob = TextBlob(str(t).decode('utf-8'))
    return map(lambda s:s.sentiment.polarity,blob.sentences)
def get_noun_phrases(t):
    blob = TextBlob(str(t).decode('utf-8'))
    return list(blob.noun_phrases)
def map_entity(e):
    pos,neg=get_entity_sentiment(e)
    return {'tag':e.tag,'entity':" ".join(e),'pos':pos,'neg':neg}
def get_entites(text):
    try:
        polyglot_text=Text(text)
        entities=polyglot_text.entities
        return map(map_entity,entities)
    except:
        return []
     
def map_to_data(tweet):
    user_name=tweet['user']['name']
    text=tweet['text']
    return {'text':text,'user':user_name,'entities':get_entites(text),'sentiment':get_sentiment(text),'noun_phrases':get_noun_phrases(text)}

In [37]:
# temp = tweets.sample(10)

In [38]:
# tweets['entities']=tweets['message'].map(get_entites)

In [40]:
def filter_str(x):
    try:
        str(x).decode('utf-8')
        return x
    except:
        return ''

In [42]:
tweets['message_2'] = tweets.message.apply(lambda x: filter_str(x))

In [44]:
tweets = tweets[tweets.message_2 != '']

In [45]:
tweets

,created_time,displayName,message,postedTime,message_2
16,2015-06-18 05:31:04,Donald Trump,Donald J. Trump ESTAS BIEN PENDEJO\nDonald J. ...,2015-06-18 05:31:04,Donald J. Trump ESTAS BIEN PENDEJO\nDonald J. ...
17,2015-07-22 18:26:43,Donald Trump,When will someone look when our economy went b...,2015-07-22 18:26:43,When will someone look when our economy went b...
18,2015-08-03 00:39:33,Donald Trump,"Hola Sr. Donald J., es un gusto saludarle, q...",2015-08-03 00:39:33,"Hola Sr. Donald J., es un gusto saludarle, q..."
19,2015-08-15 15:39:06,Donald Trump,#66 Bible is huge all 66 parts for 66 Likes..#...,2015-08-15 15:39:06,#66 Bible is huge all 66 parts for 66 Likes..#...
20,2015-08-19 18:31:22,Donald Trump,Where is the Donal Trump office in Denver? I w...,2015-08-19 18:31:22,Where is the Donal Trump office in Denver? I w...
22,2015-10-15 10:05:36,Donald Trump,MR TRUMP... how can you sit there under the es...,2015-10-15 10:05:36,MR TRUMP... how can you sit there under the es...
23,2015-10-20 14:54:45,Donald Trump,Donald J. Trump ESTAS BIEN PENDEJO\nDonald J. ...,2015-10-20 14:54:45,Donald J. Trump ESTAS BIEN PENDEJO\nDonald J. ...
24,2015-10-20 18:57:57,Donald Trump,:poop:,2015-10-20 18:57:57,:poop:
25,2015-10-21 08:08:55,Donald Trump,Donald J. Trump ESTAS BIEN PENDEJO\nDonald J. ...,2015-10-21 08:08:55,Donald J. Trump ESTAS BIEN PENDEJO\nDonald J. ...
26,2015-12-17 09:00:10,Donald Trump,All I have to say is your gonna be the first m...,2015-12-17 09:00:10,All I have to say is your gonna be the first m...


In [46]:
tweets['sentiment'] = tweets['message_2'].map(get_sentiment)

In [47]:
tweets['sentiment']

,created_time,displayName,message,postedTime,message_2,sentiment
16,2015-06-18 05:31:04,Donald Trump,Donald J. Trump ESTAS BIEN PENDEJO\nDonald J. ...,2015-06-18 05:31:04,Donald J. Trump ESTAS BIEN PENDEJO\nDonald J. ...,[0.0]
17,2015-07-22 18:26:43,Donald Trump,When will someone look when our economy went b...,2015-07-22 18:26:43,When will someone look when our economy went b...,"[-0.7, -0.7, 0.0916666666667, 0.0, 0.0, 0.1333..."
18,2015-08-03 00:39:33,Donald Trump,"Hola Sr. Donald J., es un gusto saludarle, q...",2015-08-03 00:39:33,"Hola Sr. Donald J., es un gusto saludarle, q...","[0.0, 0.0]"
19,2015-08-15 15:39:06,Donald Trump,#66 Bible is huge all 66 parts for 66 Likes..#...,2015-08-15 15:39:06,#66 Bible is huge all 66 parts for 66 Likes..#...,[0.4]
20,2015-08-19 18:31:22,Donald Trump,Where is the Donal Trump office in Denver? I w...,2015-08-19 18:31:22,Where is the Donal Trump office in Denver? I w...,"[0.0, 0.0]"
22,2015-10-15 10:05:36,Donald Trump,MR TRUMP... how can you sit there under the es...,2015-10-15 10:05:36,MR TRUMP... how can you sit there under the es...,"[-0.4, -0.6, -0.6, 0.0, 0.5]"
23,2015-10-20 14:54:45,Donald Trump,Donald J. Trump ESTAS BIEN PENDEJO\nDonald J. ...,2015-10-20 14:54:45,Donald J. Trump ESTAS BIEN PENDEJO\nDonald J. ...,[0.0]
24,2015-10-20 18:57:57,Donald Trump,:poop:,2015-10-20 18:57:57,:poop:,[0.0]
25,2015-10-21 08:08:55,Donald Trump,Donald J. Trump ESTAS BIEN PENDEJO\nDonald J. ...,2015-10-21 08:08:55,Donald J. Trump ESTAS BIEN PENDEJO\nDonald J. ...,[0.0]
26,2015-12-17 09:00:10,Donald Trump,All I have to say is your gonna be the first m...,2015-12-17 09:00:10,All I have to say is your gonna be the first m...,[0.3125]


In [62]:
dataToCorelation={}
candidates=['Bernie Sanders', 'Donald Trump', 'Hillary Clinton']
for candidate in candidates:
    candidateTweets=tweets[tweets['displayName']==candidate]
    
    dataToCorelation[candidate]=candidateTweets[['mean_sentiment','day']].groupby(['day']).mean().reset_index()

In [63]:
cols=[]
headers=[]
dates=[]
for k,v in dataToCorelation.iteritems():
    v[k]=v['mean_sentiment']
    dates.append(list(v['day']))
    cols.append(list(v[k]))
    headers.append(k+' Facebook comments')

In [64]:
import pickle
pickle.dump((cols,headers,dates),open('corelation_facebook_comments.p','wb'))

In [52]:
# tweets['noun_phrases']=tweets['message'].map(get_noun_phrases)

/usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [48]:
tweets['mean_sentiment']=tweets['sentiment'].map(lambda s:reduce(lambda x, y: x + y, s) / len(s))
tweets['postedTime_datetime']=tweets['postedTime'].map(lambda x:pd.to_datetime(str(timestring.Date(x))))
tweets['postedTime_datetime']=pd.DatetimeIndex(tweets['postedTime'])

In [49]:
tweets['displayName']=tweets['displayName'].replace('Senator Ted Cruz','Ted Cruz')
tweets['day']=tweets['postedTime_datetime'].map(lambda x:x.date())
tweets['week']=tweets['day'].map(lambda x:x.isocalendar()[1])

In [50]:
tweets.to_csv('datasets/facebook_comments_with_desc.csv')

In [52]:
candidates=set(tweets['displayName'])

In [54]:
candidates

{'Bernie Sanders', 'Donald Trump', 'Hillary Clinton'}

In [55]:
data={}
traces=[]

for candidate in candidates:
    candidateTweets=tweets[tweets['displayName']==candidate]
    sentimentData=candidateTweets[['mean_sentiment','day']]
    data[candidate]=sentimentData
    sentimentData['day']=pd.to_datetime(sentimentData['day'])
    sentimentData.set_index(sentimentData['day'],inplace=True)
    sentimentDataToSave=sentimentData.resample("1d").mean().fillna(0).rolling(window=7, win_type='triang').mean()
    print candidate+': '+str(sentimentDataToSave['mean_sentiment'].std())
    sentimentDataToSave.to_csv('facebook_comments_sentiment/'+candidate+'.csv')
    traces.append(
        go.Scatter(
            x = sentimentDataToSave.index,
            y = sentimentDataToSave['mean_sentiment'],
            mode = 'lines',
            name = candidate
        )
    )
    

Donald Trump: 0.0595450695191
Bernie Sanders: 0.0311121929151
Hillary Clinton: 0.0443255653774


/usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [56]:
import plotly.plotly as py
import plotly.graph_objs as go

py.sign_in('RomanBartusiak', 'c10jv3qe0b')
py.iplot(traces, filename='scatter-mode-facebook-comments')
   

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.



In [95]:
downloader.download('ner2.sco',download_dir='/root/polyglot_data')

[polyglot_data] Error loading ner2.sco: Package u'ner2.sco' not found
[polyglot_data]     in index


False

NameError: name 'tweet' is not defined

In [139]:
entities_count = {}

In [126]:
# {for k, v tweets['entities']}

In [140]:
for row_id, tweet in tweets.iterrows():
    for ent in tweet['entities']:
        e = ent['entity']
        if e not in entities_count:
            entities_count.update({e: 1})
        else:
            entities_count[e] += 1

In [181]:
entities_count = {k.strip().encode('ascii','ignore'):v for k, v in entities_count.iteritems() if len(k) > 2 and v > 4}

In [182]:
entities_count

{': New Hampshire': 6,
 'ABC': 47,
 'ACA': 6,
 'AMERICA': 32,
 'AMERICA SAFE': 5,
 'Aberdeen': 6,
 'Adam': 16,
 'Affordable': 12,
 'Afghanistan': 10,
 'Aiken': 5,
 'Alabama': 34,
 'Alaska': 10,
 'Alberto Gonzales': 5,
 'America': 757,
 "America's": 7,
 'American': 60,
 'American Airlines Center': 5,
 'Ames': 16,
 'Ames , Iowa': 6,
 'Amir Hekmati': 5,
 'Amnesty': 6,
 'Anderson': 8,
 'Antonin Scalia': 7,
 'Apple': 7,
 'Arizona': 49,
 'Arkansas': 12,
 'Arnold': 5,
 'Assad': 6,
 'Atlanta': 24,
 'Austin': 5,
 'BET': 8,
 'BUSH': 7,
 'Baltimore': 11,
 'Barack Obama': 18,
 'Barbara Walters': 5,
 'Beijing': 8,
 'Ben': 11,
 'Ben Carson': 22,
 'Benghazi': 17,
 'Bern': 31,
 'Bernie': 314,
 'Bernie Sanders': 331,
 "Bernie Sanders's": 7,
 'BernieSanders': 25,
 'Biden': 9,
 'Big Pharma': 9,
 'Bill': 12,
 'Bill Clinton': 8,
 "Bill O'Reilly": 9,
 'Birmingham': 8,
 'Bloomberg': 8,
 'Bluffs , Iowa': 5,
 'Bob': 8,
 'Bob Vander Plaats': 7,
 'Bobby Knight': 7,
 'Boehner': 7,
 'Boston': 17,
 'Boulder': 5,
 '

In [183]:
len(entities_count)

689

In [136]:
for row_id, tweet in tweets.iterrows():
    for ent in tweet['entities']:
        if ent['entity'] not in entities:
            entities[ent['entity']].update({})

In [148]:
!pip2 install wordcloud

/usr/local/lib/python2.7/dist-packages/pip/_vendor/requests/packages/urllib3/util/ssl_.py:315: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#snimissingwarning.
  SNIMissingWarning
/usr/local/lib/python2.7/dist-packages/pip/_vendor/requests/packages/urllib3/util/ssl_.py:120: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
  Using cached wordcloud-1.2.1.tar.gz
  Running setup.py bdist_wheel for wordcloud ... - \ done
  Stored in directory: /root/.cache/pip

In [149]:
from wordcloud import WordCloud

In [184]:
wordcloud = WordCloud(background_color="white").generate_from_frequencies(entities_count)

ValueError: could not convert string to float: y

In [ ]:
# def draw_wc_filtered(df, top_n=10, save=False, agg_col=u'Quantity'): 
#     df = pd.DataFrame(df.groupby(u'Group').sum()[agg_col]).sort(
#         agg_col, ascending=False).ix[:top_n].reset_index()
#     df.Group = df.Group.apply(lambda x: x.split(' - ')[1])
#     counts = [((x[1]['Group'].decode('utf-8'), x[1][agg_col])) for x in df.iterrows()]
# #     print counts
#     wordcloud = WordCloud(background_color="white").generate_from_frequencies(counts)
#     plt.figure(figsize=(9, 12))
#     plt.imshow(wordcloud)
#     plt.axis("off")
#     plt.show()
#     if save:
#         plt.savefig('wc/{}.png'.format(save))